In [1]:
from json import load

from ecomm import MLInterface
from httpx import Response
from pandas import read_excel, read_feather, merge, DataFrame
from rich import print as pprint
from clone import Clanador

In [2]:
from pandas import options
options.display.max_columns = None

In [3]:
def get_number_photo(lista_name_photo: str) -> list[str]:
    return [name.split('_')[-1:][0].replace('.jpg', '') for name in lista_name_photo]

In [4]:
headers = MLInterface(1)._headers()

In [5]:
with open('./data/sale_terms.json', 'r', encoding='utf-8') as fp:
    sale_terms = load(fp)

In [7]:
df_photos: DataFrame = read_feather(
    '../data/conferencia_fotos_monroe.feather')

In [8]:
df_photos = df_photos.query('~mlb.isna()').reset_index(drop=True).copy()

In [9]:
df_photos['number_photo'] = get_number_photo(
    df_photos.path_file_photo.fillna('0'))

In [14]:
df_clona_genuino = read_excel('../data/monroe.xlsx', dtype=str)

In [15]:
df_clona_genuino = df_clona_genuino.set_axis(
    df_clona_genuino.columns.str.lower(), axis=1).copy()

In [17]:
df_clona_genuino.loc[:, 'sku_certo'] = df_clona_genuino.sku_certo.str.strip()

In [18]:
df_clona_copy: DataFrame = merge(
    df_clona_genuino,
    df_photos,
    on='mlb',
    how='left'
)

In [19]:
df_clona_copy = df_clona_copy.query(
    'clona == "1" and (verifeid_photo and pegar_foto)').reset_index(drop=True).copy()

In [20]:
clonagem = Clanador(
    headers=headers,
    sales_terms=sale_terms
)

In [21]:
df_clona_copy = df_clona_copy.sort_values(['mlb', 'number_photo']).reset_index(drop=True)

In [22]:
clonagem.read_product_siac()

In [23]:
for mlb in df_clona_copy.mlb.unique():
    row: DataFrame = df_clona_copy.query('mlb == @mlb')
    sku: str = row['sku_certo'].unique()[0]
    codpro: str = clonagem.convert_sku_to_codpro(
        sku=sku
    )[0]
    display(row)

    clonagem.df_siac_filter(
        codpro_produto=codpro
    )
    break

,mlb,gtin_ml,gtin_siac,gtin_fuzzy,mpn_ml,mpn_siac,mpn_fuzzy,sku_ml,sku_siac,sku_certo,sku_fuzzy,numero_original_ml,numero_original_siac,numero_original_fuzzy,marca_ml,marca_siac,marca_fuzzy,lista_url_anuncios,lista_mlb,soma_fuzzy,clona,verifeid_photo,pegar_foto,path_file_photo,number_photo
0,MLB1107990585,4011442606862,4047026263862,69,6204,62046,89,LT60435 LT60436,LT60456,K_087449_087450,61,RUV925008RUV925009,9295080059,57,MONROE AXIOS,MONROE,67,https://produto.mercadolivre.com.br/MLB-110799...,NaN,343,1,True,True,MLB1107990585_PHOTOID838075-MLB49782381300_042...,0


In [24]:
mlb_info: tuple[Response, Response] = clonagem.mlb_infos(
    item_id_ml=row.loc[0, 'mlb']
)

In [25]:
caminho_fotos = '../photo_validation/out_files_photos/%s'

In [27]:
[caminho_fotos%f'monroe/{name_photo}' for name_photo in row['path_file_photo']]

['../photo_validation/out_files_photos/monroe/MLB1107990585_PHOTOID838075-MLB49782381300_042022_0.jpg']

In [28]:
clonagem.gerar_payload_cadastro(
    list_path_img=[
        caminho_fotos % f'monroe/{name_photo}' for name_photo in row['path_file_photo']],
    sku=sku
)

{
    'title': None,
    'category_id': None,
    'official_store_id': None,
    'price': None,
    'available_quantity': None,
    'currency_id': 'BRL',
    'buying_mode': 'buy_it_now',
    'condition': 'new',
    'listing_type_id': None,
    'sale_terms': None,
    'pictures': None,
    'attributes': None,
    'variations': None,
    'video_id': 'e4lB8zcfDt8'
}

<Response [200 OK]>

c:\Users\jeferson.lopes\Documents\wk-python\match-market-ads\clonagem\clone.py:135: FutureWarning: Possible nested set at position 1
  'value_name': sub(


In [30]:
corpo_cadastro: dict = clonagem.corpo_clonagem

In [31]:
pprint(corpo_cadastro)

{
    'title': 'Par Bieleta Estabilizadora Dianteira Bmw 318i 320i 05/11 E90',
    'category_id': 'MLB193476',
    'official_store_id': 5329,
    'price': 51.92,
    'available_quantity': 0,
    'currency_id': 'BRL',
    'buying_mode': 'buy_it_now',
    'condition': 'new',
    'listing_type_id': 'gold_special',
    'sale_terms': [
        {
            'id': 'WARRANTY_TYPE',
            'name': 'Tipo de garantia',
            'value_id': '2230279',
            'value_name': 'Garantia de fábrica',
            'value_struct': None,
            'values': [{'id': '2230279', 'name': 'Garantia de fábrica', 'struct': None}],
            'value_type': 'list'
        },
        {
            'id': 'WARRANTY_TIME',
            'name': 'Tempo de garantia',
            'value_id': None,
            'value_name': '3 meses',
            'value_struct': {'number': 3, 'unit': 'meses'},
            'values': [{'id': None, 'name': '3 meses', 'struct': {'number': 3, 'unit': 'meses'}}],
            'value_type': 'number_unit'
        }
    ],
    'pictures': [{'id': '890247-MLB77706886874_072024'}],
    'attributes': [
        {'id': 'BRAND', 'name': 'Marca', 'value_name': 'AXIOS'},
        {'id': 'GTIN', 'name': 'Código universal de produto', 'value_name': None},
        {
            'id': 'INMETRO_CERTIFICATION_REGISTRATION_NUMBER',
            'name': 'Número de registro/certificação INMETRO',
            'value_id': None,
            'value_name': 'LT60435 / LT60436',
            'values': [{'id': None, 'name': 'LT60435 / LT60436', 'struct': None}],
            'value_type': 'string'
        },
        {
            'id': 'ITEM_CONDITION',
            'name': 'Condição do item',
            'value_id': '2230284',
            'value_name': 'Novo',
            'values': [{'id': '2230284', 'name': 'Novo', 'struct': None}],
            'value_type': 'list'
        },
        {
            'id': 'LENGTH',
            'name': 'Comprimento',
            'value_id': None,
            'value_name': '29 cm',
            'values': [{'id': None, 'name': '29 cm', 'struct': {'number': 29, 'unit': 'cm'}}],
            'value_type': 'number_unit'
        },
        {
            'id': 'MODEL',
            'name': 'Modelo',
            'value_id': None,
            'value_name': 'BIELETA ESTABILIZADORA',
            'values': [{'id': None, 'name': 'BIELETA ESTABILIZADORA', 'struct': None}],
            'value_type': 'string'
        },
        {'id': 'MPN', 'name': 'MPN', 'value_name': 'LT60435'},
        {'id': 'OEM', 'name': 'Código OEM', 'value_name': 'None'},
        {
            'id': 'PACKAGE_HEIGHT',
            'name': 'Altura da embalagem',
            'value_id': None,
            'value_name': '11.5 cm',
            'values': [{'id': None, 'name': '11.5 cm', 'struct': {'number': 11.5, 'unit': 'cm'}}],
            'value_type': 'number_unit'
        },
        {
            'id': 'PACKAGE_LENGTH',
            'name': 'Comprimento da embalagem',
            'value_id': None,
            'value_name': '46.5 cm',
            'values': [{'id': None, 'name': '46.5 cm', 'struct': {'number': 46.5, 'unit': 'cm'}}],
            'value_type': 'number_unit'
        },
        {
            'id': 'PACKAGE_WEIGHT',
            'name': 'Peso da embalagem',
            'value_id': None,
            'value_name': '1070 g',
            'values': [{'id': None, 'name': '1070 g', 'struct': {'number': 1070, 'unit': 'g'}}],
            'value_type': 'number_unit'
        },
        {
            'id': 'PACKAGE_WIDTH',
            'name': 'Largura da embalagem',
            'value_id': None,
            'value_name': '15.5 cm',
            'values': [{'id': None, 'name': '15.5 cm', 'struct': {'number': 15.5, 'unit': 'cm'}}],
            'value_type': 'number_unit'
        },
        {'id': 'PART_NUMBER', 'name': 'Número de peça', 'value_name': 'LT60435'},
        {
            'id': 'POSITION',
            'name': 'Posição',
            'value_id': '13701104',
         